In [7]:
"air_this_2132".split("_",-1)

['air', 'this', '2132']

In [1]:
import sys
sys.path.append("/Users/sebastiaanscholten/Documents/speech2image-master/preprocessing")
import os
import json
import pickle
from visual_features import vis_feats
from audio_features import audio_features
from pathlib import Path
import tables
import pandas as pd

ModuleNotFoundError: No module named 'torch'

In [5]:
def iterate_data(h5_file):
    for x in h5_file.root:
        yield x
path ="/Users/sebastiaanscholten/Documents/speech2image-master/vgsexperiments/experiments/data/flickr_features_resnetnormal_final.h5"

resnetfeatures = tables.open_file(path, mode="r+")
resnetfeatures = [node for node in iterate_data(resnetfeatures)]
resnetobjects = pd.DataFrame()

for node in resnetfeatures:
    filename = node._v_name
    objects = sum(node.resnetnormal._f_list_nodes()[0][:])
    resnetobjects = resnetobjects.append({"Filename":filename,"Objects":objects},ignore_index=True)

IndexError: Too many indices for object '/flickr_1000268201_693b08cb0e/resnetnormal/flickr_1000268201_693b08cb0e'

In [24]:
resnetfeatures[2].resnetnormal._f_list_nodes()

[/flickr_1002674143_1b742ab4b8/resnetnormal/flickr_1002674143_1b742ab4b8 (Array()) ''
   atom := Float32Atom(shape=(), dflt=0.0)
   maindim := 0
   flavor := 'numpy'
   byteorder := 'little'
   chunkshape := None]

In [29]:
import os
import json
import pickle
from visual_features import vis_feats
from audio_features import audio_features
from pathlib import Path
import tables
# path to the flickr audio, caption and image files
# audio_path = os.path.join('/Users/sebastiaanscholten/Documents/speech2image-master/preprocessing/testfolder2/test/wavs')
# img_path = os.path.join('/Users/sebastiaanscholten/Documents/speech2image-master/preprocessing/testfolder2/test/jpgs')
img_path = os.path.join('/Volumes/Schnellheit/ThesisDatasets/Flickr_datasets/Flickr8k_Dataset/Flicker8k_Dataset')
audio_path = os.path.join('/Volumes/Schnellheit/ThesisDatasets/Flickr_datasets/flickr_audio/wavs')

text_path = os.path.join('/Users/sebastiaanscholten/Documents/speech2image-master/preprocessing/testfolder2/test/dataset.json')
# save the resulting feature file here
data_loc = os.path.join('resnetfeatures/flickr_features_resnetnormal7.h5')

# some bools in case only some new features needs to be added
vis = ["resnetnormal"]
speech = ['mfcc']
text = True

def load_obj(loc):
    with open(loc + '.pkl', 'rb') as f:
        return pickle.load(f)

# list the img and audio directories
audio = os.listdir(audio_path)
imgs = os.listdir(img_path)

# strip the files to their basename and remove file extension
imgs_base = [x.split('.')[0] for x in imgs]
audio_base = [x.split('.')[0] for x in audio]

# create a dictionary with the common part in the images and audio filenames as
# keys pointing to the image and caption file names
img_audio = {}
no_cap = []
for im in imgs_base:
    temp = []
    for cap in audio_base:
        # match the images with the appropriate captions
        if im in cap:
           temp.append(cap + '.wav')
        # quit the loop early if all captions are found
        if len (temp) == 5:
            break
    # make a dictionary entry with the basename of the files as key, containing
    # the full names of the img and audio files
    if temp:
        img_audio[im] = im + '.jpg', temp
    else:
        # keep track of images without captions
        no_cap.append(im)
# we need to append something to the flickr files names because pytable group names cannot start
# with integers.
append_name = 'flickr_'

# if the output file does not exist yet, create it
if not Path(data_loc).is_file():
    # create h5 output file for preprocessed images and audio
    output_file = tables.open_file(data_loc, mode='a')
    # create the h5 file to hold all image and audio features. This will fail if they already excist such
    # as when you run this file to append new features to an excisting feature file
    for x in img_audio:
        try:
            # one group for each image file which will contain its vgg16 features and audio captions
            output_file.create_group("/", append_name + x.split('.')[0])
        except:
            continue
# else load an existing file to append new features to
else:
    output_file = tables.open_file(data_loc, mode='a')

#list all the nodes
node_list = output_file.root._f_list_nodes()

# create the visual features for all images
for x in vis:
    vis_feats(img_path, output_file, append_name, img_audio, node_list, x)

processing file visual:1
processing file visual:2
processing file visual:3


KeyboardInterrupt: 

In [30]:
import os

import PIL.Image
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

In [31]:
def resnetnormal():
    # initialise a pretrained model, see torch docs for the availlable pretrained models. Torch will download
    # the weights automatically
    model = models.resnet152(pretrained = True)

    # remove the final layer from the classifier. This may need to be adapted somewhat for different models
    # because of the module names
    return model

In [118]:
def prep_resize(im, model):
    for p in model.parameters():
        p.requires_grad = False
    model.eval()
        
    # some functions such as taking the ten crop (four corners, center and horizontal flip) normalise and resize.
    tens = transforms.ToTensor()
    normalise = transforms.Normalize(mean = [0.485,0.456,0.406], 
                                     std = [0.229, 0.224, 0.225])
    resize = transforms.Resize((224,224), PIL.Image.ANTIALIAS)
    
    im = resize(im)
    im = normalise(tens(im))
    if torch.cuda.is_available():
        im = im.cuda()
    # there are some grayscale images in mscoco that the vgg and resnet networks
    # wont take
    if not im.size()[0] == 3:
        im = im.expand(3, im.size()[2], im.size()[3])
    activations = model(im.unsqueeze(0))
    return activations.squeeze()

def prep_tencrop(im, model):
    for p in model.parameters():
    	p.requires_grad = False
    model.eval()
        
    # some functions such as taking the ten crop (four corners, center and horizontal flip) normalise and resize.
    tencrop = transforms.TenCrop(224)
    tens = transforms.ToTensor()
    normalise = transforms.Normalize(mean = [0.485,0.456,0.406], 
                                     std = [0.229, 0.224, 0.225])
    resize = transforms.Resize(256, PIL.Image.ANTIALIAS)
    
    im = tencrop(resize(im))
    im = torch.cat([normalise(tens(x)).unsqueeze(0) for x in im])
    if torch.cuda.is_available():
        im = im.cuda()
    # there are some grayscale images in mscoco that the vgg and resnet networks
    # wont take
    if not im.size()[1] == 3:
        im = im.expand(im.size()[0], 3, im.size()[2], im.size()[3])
    activations = model(im)
    return activations.mean(0).squeeze()

In [170]:
model = resnetnormal()
testimage = PIL.Image.open('/Volumes/Schnellheit/ThesisDatasets/Flickr_datasets/Flickr8k_Dataset/Flicker8k_Dataset/3718892835_a3e74a3417.jpg')
activations = prep_tencrop(testimage,model)

In [171]:
activations[151]

tensor(1.6710)

In [172]:
sigmoid = torch.nn.functional.sigmoid(activations)

In [175]:
[i for i,v in enumerate(sigmoid) if v > 0.9999]

[444, 523, 870, 880]

In [174]:
sum(sigmoid>0.9999)

tensor(4)